## LLM as judge

Rate voter guides

How the WA voter pamphlet works:
https://washingtonstatestandard.com/2024/07/01/how-the-washington-state-voter-guide-became-a-110-year-democratic-tradition/



Best practices: https://www.eac.gov/sites/default/files/2024-07/Best_practices_for_designing_effective_voter_ed_materials_508.pdf


In [1]:
GEMINI="gemini-2.0-flash"
OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please specify the OPENAI_API_KEY access token in keys.env file"

In [2]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio
nest_asyncio.apply()
def zero_shot(model_id, prompt: str) -> str:
    from pydantic_ai import Agent
    agent = Agent(model_id)
    result = agent.run_sync(prompt)
    return (result.data)

## Text of voter pamphlet

Source of 2022 initiatives text:
https://www.sos.wa.gov/sites/default/files/2022-05/Voters%2520Pamphlet%25202016.pdf

In [3]:
argument_for = """
Washington State has taken important steps to keep guns
out of dangerous hands. But there are still gaps in our laws
that make it hard to keep guns away from people threatening
violence against themselves or others. We know that the
majority of mass shooters and individuals who attempt suicide
show signs of their intentions, but current law leaves families
and law enforcement - often first to see those warning signs -
unable to take life-saving action.

**Initiative 1491: Empower Families, Prevent Gun Violence**
Initiative 1491 empowers families and law enforcement to
prevent tragedy -- giving them a chance to remove guns from
a dangerous situation when they know someone is a threat
to themselves or others. Parents of shooters at Isla Vista,
Seattle’s Cafe Racer, and other tragedies have said they could
have used this type of law to prevent senseless violence.
Initiative 1491 would also expand protections that keep guns
out of the hands of domestic abusers. Similar laws in other
states have been shown to prevent some suicides.

**Initiative 1491: Respect Due Process**
Initiative 1491 closely follows existing process for other civil
protection orders. Both parties may present evidence in court.
A judge determines whether evidence of danger is sufficient
and issues an order, effective for one year. There are criminal
penalties for false petitions.

**Initiative 1491: Community Support**
Endorsed by Washington State Public Health Association,
League of Women Voters, Faith Action Network, Everytown
for Gun Safety, law enforcement, domestic violence experts,
gun owners, and gun violence survivors.

**Rebuttal of argument against**
Initiative 1491 fills a critical need in Washington’s proven,
established protection order system. It simply gives families
a tool to save lives— keeping guns from loved ones who
are likely to use them for violence to themselves or others.
Initiative 1491 is a targeted, tested way to keep guns out of
dangerous hands and respect due process—endorsed by
mental health professionals, law enforcement and suicide
prevention advocates
"""

In [4]:
argument_against = """
**I-1491 Duplicates Existing Laws**
I-1491 disregards existing state laws that already require
treatment and restriction of potentially dangerous individuals.
I-1491 doesn’t require evaluation, treatment, or monitoring
and does nothing to address underlying issues. Recently implemented laws actually provide early detection and intervention of persons at danger to themselves or others.

**Stigmatizes Mental Illness**
I-1491 associates mental illness with mass shootings and violent crime. Statistics show that only 3%-5% of violent acts
are committed by people with serious mental illness. The vast
majority of people with mental illness are not violent and are
ten times more likely to be victims of violent crime than the
general population.

**Violates Rights**
A broadly defined set of people, including former roommates
and police, can file a petition against you. Due process is undermined by allowing immediate ex parte orders; hearings and
judgments without notice to the accused person. The definition of “Extreme Risk” is unclear. A judge can issue an order
based on arbitrary factors and reported behaviors including
simply purchasing a gun legally. To be released from an order,
a person must prove he/she is not a danger to themselves or
others and pay for the tremendous cost of their own defense.

**Gives False Sense of Security**
There is no evidence that such orders reduce mass shootings
and violent crime.
Restrictions on firearm ownership should not be based on
ideological agendas manipulating public fears and misconceptions about gun violence. I-1491 is a targeted, discriminatory abridgement of Second Amendment rights. Vote No!

**Rebuttal of argument for**
Ineffective! We all want to reduce tragedy, but I-1491 doesn’t
include treatment of allegedly dangerous people, and
doesn’t remove other dangerous items (vehicles, knives…).
Misdirected! I-1491 ignores that 95-97% of violent crimes are
not related to mental illness. Deceptive! In Isla Vista, the parents
told police they “found it difficult to believe their son either
owned weapons or would actually hurt anyone.” Unintended
consequences! Confiscating firearms doesn’t make someone
stable, it makes them mad.
"""

## LLM-as-Judge

the scoring rubric

In [5]:
scoring_criteria = """
- Centers the voter: easy for a voter reading this to understand how the initiative will affect them. Consider voters at different socioeconomic status, and education levels.
- Organizes information as a pyramid: the most essential information is presented first, and inessential details and explanations last.
- Understandable: Uses plain language, simple sentences, and minimizes jargon.
- Clarity: The call to action is clear. Why should the voter vote for/against this initiative?
- Caters to undecided: Provide additional information that may sway undecided voters for/against the cause’s direction. This might be endorsements from authoritative/neutral groups or a specific comparison to the competition.
"""

In [6]:
def create_llm_as_judge_prompt(argument: str) -> str:
    return f"""
    You will be given an argument for or against a Washington State initiative
    which will appear in the voter pamphlet that is mailed to all households.

    Provide a score in the range 1-5 for each of the following criteria:
    {scoring_criteria}

    For each score, provide a brief justification.

    **Argument**:
    {argument}

    **Score**":
    """

In [8]:
rubric = zero_shot(OPENAI, create_llm_as_judge_prompt(argument_for))
print(rubric)

**Criteria Scores and Justifications:**

1. **Centers the voter: 4**
   - The argument clearly explains how Initiative 1491 will empower families and law enforcement to take action against potential threats, which is relevant to voters of varying backgrounds. However, while it highlights the importance of the initiative in preventing violence, it could more explicitly address how this law impacts the everyday lives of average citizens, especially those at lower socioeconomic levels.

2. **Organizes information as a pyramid: 5**
   - The essential information is presented first, with a clear statement about what Initiative 1491 aims to do, followed by support from tragic incidents that illustrate the need. Further explanations about the process and community support are provided afterward, ensuring that crucial details are prioritized.

3. **Understandable: 4**
   - The language is mostly plain, and the sentences are straightforward, which makes the argument accessible. However, some re

In [9]:
rubric = zero_shot(OPENAI, create_llm_as_judge_prompt(argument_against))
print(rubric)

**Centers the voter: 3/5**
The argument discusses how I-1491 could affect individuals' rights, mental health stigma, and due process. However, it doesn't explicitly connect these points to how the initiative impacts a wide range of voters from different backgrounds. While it mentions mental illness and rights, it would benefit from clearer implications for everyday voters, especially those who may not have immediate concerns about firearms or mental health.

**Organizes information as a pyramid: 4/5**
The argument effectively presents the most critical points at the beginning, detailing how I-1491 duplicates existing laws and stigmatizes mental illness. However, the flow could be improved by prioritizing the specific rights violations next, followed by the implications of the false sense of security. The rebuttal section is valuable but could have been integrated earlier in the argument to bolster immediate rebuttal to proponents of I-1491.

**Understandable: 4/5**
The language used is